In [146]:
# Load data
import pandas as pd
import numpy as np
import nltk
url = "SMSSpamCollection.txt"
df = pd.read_csv(url, sep='\t', header=None )
df.rename(columns={0:'type',1:'mail'}, inplace=True)
df


,type,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [147]:
df['minuscule']=df['mail'].str.lower()
df

,type,mail,minuscule
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro..."
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?,will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...


In [148]:
# tokenizer avec RE (regular expressions)
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')

tokenizer = RegexpTokenizer(r"[a-zA-Z]\w+\'?\w*")

df['token'] = df['minuscule'].apply(lambda x: tokenizer.tokenize(x))
df


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,type,mail,minuscule,token
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup..."
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, don't, think, he, goes, to, usf, he, liv..."
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...,"[this, is, the, nd, time, we, have, tried, con..."
5568,ham,Will ü b going to esplanade fr home?,will ü b going to esplanade fr home?,"[will, going, to, esplanade, fr, home]"
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s...","[pity, was, in, mood, for, that, so, any, othe..."
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...,"[the, guy, did, some, bitching, but, acted, li..."


In [149]:

from nltk.corpus import stopwords
nltk.download('stopwords')
# Supprimer les stop words
stop = stopwords.words('english')

df['without_stopwords']=df['token'].apply(lambda x: [word for word in x if word not in stop])
df


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,type,mail,minuscule,token,without_stopwords
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, g..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, oni]","[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup...","[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[dun, say, so, early, hor, already, then, say]","[dun, say, early, hor, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, don't, think, he, goes, to, usf, he, liv...","[nah, think, goes, usf, lives, around, though]"
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...,"[this, is, the, nd, time, we, have, tried, con...","[nd, time, tried, contact, pound, prize, claim..."
5568,ham,Will ü b going to esplanade fr home?,will ü b going to esplanade fr home?,"[will, going, to, esplanade, fr, home]","[going, esplanade, fr, home]"
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s...","[pity, was, in, mood, for, that, so, any, othe...","[pity, mood, suggestions]"
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...,"[the, guy, did, some, bitching, but, acted, li...","[guy, bitching, acted, like, interested, buyin..."


In [150]:

from nltk.stem import PorterStemmer


# stemmer notre list

stemmer = PorterStemmer()
df['PorterStemmer'] = df['without_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
df

,type,mail,minuscule,token,without_stopwords,PorterStemmer
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, g...","[go, jurong, point, crazi, avail, bugi, great,..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, oni]","[ok, lar, joking, wif, oni]","[ok, lar, joke, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup...","[free, entry, wkly, comp, win, fa, cup, final,...","[free, entri, wkli, comp, win, fa, cup, final,..."
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[dun, say, so, early, hor, already, then, say]","[dun, say, early, hor, already, say]","[dun, say, earli, hor, alreadi, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, don't, think, he, goes, to, usf, he, liv...","[nah, think, goes, usf, lives, around, though]","[nah, think, goe, usf, live, around, though]"
...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...,"[this, is, the, nd, time, we, have, tried, con...","[nd, time, tried, contact, pound, prize, claim...","[nd, time, tri, contact, pound, prize, claim, ..."
5568,ham,Will ü b going to esplanade fr home?,will ü b going to esplanade fr home?,"[will, going, to, esplanade, fr, home]","[going, esplanade, fr, home]","[go, esplanad, fr, home]"
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s...","[pity, was, in, mood, for, that, so, any, othe...","[pity, mood, suggestions]","[piti, mood, suggest]"
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...,"[the, guy, did, some, bitching, but, acted, li...","[guy, bitching, acted, like, interested, buyin...","[guy, bitch, act, like, interest, buy, someth,..."


In [151]:

df['clean'] = df['without_stopwords'].apply(lambda x: " ".join(x))
df

,type,mail,minuscule,token,without_stopwords,PorterStemmer,clean
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, g...","[go, jurong, point, crazi, avail, bugi, great,...",go jurong point crazy available bugis great wo...
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, oni]","[ok, lar, joking, wif, oni]","[ok, lar, joke, wif, oni]",ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup...","[free, entry, wkly, comp, win, fa, cup, final,...","[free, entri, wkli, comp, win, fa, cup, final,...",free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[dun, say, so, early, hor, already, then, say]","[dun, say, early, hor, already, say]","[dun, say, earli, hor, alreadi, say]",dun say early hor already say
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, don't, think, he, goes, to, usf, he, liv...","[nah, think, goes, usf, lives, around, though]","[nah, think, goe, usf, live, around, though]",nah think goes usf lives around though
...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...,"[this, is, the, nd, time, we, have, tried, con...","[nd, time, tried, contact, pound, prize, claim...","[nd, time, tri, contact, pound, prize, claim, ...",nd time tried contact pound prize claim easy c...
5568,ham,Will ü b going to esplanade fr home?,will ü b going to esplanade fr home?,"[will, going, to, esplanade, fr, home]","[going, esplanade, fr, home]","[go, esplanad, fr, home]",going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s...","[pity, was, in, mood, for, that, so, any, othe...","[pity, mood, suggestions]","[piti, mood, suggest]",pity mood suggestions
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...,"[the, guy, did, some, bitching, but, acted, li...","[guy, bitching, acted, like, interested, buyin...","[guy, bitch, act, like, interest, buy, someth,...",guy bitching acted like interested buying some...


In [152]:
# data
import pandas as pd


# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler

# Pipeline and model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
# Score of models
from sklearn.metrics import accuracy_score

# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, RobustScaler
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer

In [153]:
df 

,type,mail,minuscule,token,without_stopwords,PorterStemmer,clean
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, g...","[go, jurong, point, crazi, avail, bugi, great,...",go jurong point crazy available bugis great wo...
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, oni]","[ok, lar, joking, wif, oni]","[ok, lar, joke, wif, oni]",ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup...","[free, entry, wkly, comp, win, fa, cup, final,...","[free, entri, wkli, comp, win, fa, cup, final,...",free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[dun, say, so, early, hor, already, then, say]","[dun, say, early, hor, already, say]","[dun, say, earli, hor, alreadi, say]",dun say early hor already say
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, don't, think, he, goes, to, usf, he, liv...","[nah, think, goes, usf, lives, around, though]","[nah, think, goe, usf, live, around, though]",nah think goes usf lives around though
...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...,"[this, is, the, nd, time, we, have, tried, con...","[nd, time, tried, contact, pound, prize, claim...","[nd, time, tri, contact, pound, prize, claim, ...",nd time tried contact pound prize claim easy c...
5568,ham,Will ü b going to esplanade fr home?,will ü b going to esplanade fr home?,"[will, going, to, esplanade, fr, home]","[going, esplanade, fr, home]","[go, esplanad, fr, home]",going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s...","[pity, was, in, mood, for, that, so, any, othe...","[pity, mood, suggestions]","[piti, mood, suggest]",pity mood suggestions
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...,"[the, guy, did, some, bitching, but, acted, li...","[guy, bitching, acted, like, interested, buyin...","[guy, bitch, act, like, interest, buy, someth,...",guy bitching acted like interested buying some...


In [154]:
# Train test split

X = df['clean']
y = df['type']
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
X

0       go jurong point crazy available bugis great wo...
1                                   ok lar joking wif oni
2       free entry wkly comp win fa cup final tkts st ...
3                           dun say early hor already say
4                  nah think goes usf lives around though
                              ...                        
5567    nd time tried contact pound prize claim easy c...
5568                              going esplanade fr home
5569                                pity mood suggestions
5570    guy bitching acted like interested buying some...
5571                                       rofl true name
Name: clean, Length: 5572, dtype: object

In [155]:
column_text=df.columns[-1]
column_text

'clean'

In [156]:
vectorizer2 = CountVectorizer(analyzer='word', strip_accents ='unicode' )
X2 = vectorizer2.fit_transform(X_train)
X2.shape
#vectorizer2.get_feature_names_out()

# itvectoriser

(4457, 6959)

In [157]:
print(vectorizer2.get_feature_names_out()[:100])

['a21' 'a30' 'aa' 'aah' 'aaniye' 'aathi' 'ab' 'abbey' 'abdomen' 'abeg'
 'abel' 'aberdeen' 'abi' 'ability' 'abiola' 'abj' 'able' 'abnormally'
 'abroad' 'absence' 'absolutly' 'abt' 'abta' 'aburo' 'abuse' 'abusers'
 'ac' 'acc' 'accenture' 'accept' 'access' 'accessible' 'accidant'
 'accident' 'accidentally' 'accommodationvouchers' 'accomodate'
 'accomodations' 'accordin' 'accordingly' 'account' 'accounting'
 'accounts' 'accumulation' 'achan' 'ache' 'achieve' 'acid'
 'acknowledgement' 'acl03530150pm' 'acnt' 'aco' 'across' 'acted' 'actin'
 'acting' 'action' 'activ8' 'activate' 'active' 'activities' 'actor'
 'actual' 'actually' 'ad' 'adam' 'add' 'addamsfa' 'added' 'addicted'
 'addie' 'adding' 'address' 'adewale' 'adjustable' 'admin' 'administrator'
 'admirer' 'admission' 'admit' 'adore' 'adoring' 'adress' 'adrian'
 'adrink' 'ads' 'adsense' 'adult' 'adults' 'advance' 'advice' 'advise'
 'advising' 'ae' 'aeronautics' 'aeroplane' 'afew' 'affair' 'affairs'
 'affection']


In [158]:
# Transformation of textual variables
transfo_text = Pipeline(steps=[
    #('bow', CountVectorizer(decode_error='ignore', analyzer='word', ngram_range=(2, 2)))
    ('bow', CountVectorizer(decode_error='ignore', analyzer='char_wb',strip_accents='unicode'))
    # ('bow', TfidVectorizer(decode_error='ignore', analyzer='char_wb', ngram_range=(2, 2),strip_accents='unicode'))
])

# Class ColumnTransformer : apply alls steps on the whole dataset
preparation = ColumnTransformer(
    transformers=[
        ('data_bow', transfo_text , column_text)
    ])
preparation

# svc

ColumnTransformer(transformers=[('data_bow',
                                 Pipeline(steps=[('bow',
                                                  CountVectorizer(analyzer='char_wb',
                                                                  decode_error='ignore',
                                                                  strip_accents='unicode'))]),
                                 'clean')])

In [159]:
# Creation of model : a ready to use pipeline for ML process

classifier =ComplementNB()
# classifier = MultinomialNB()


In [160]:
model_lm = Pipeline([
    ('vectorizer', transfo_text),
    ('classifier', classifier)
])

In [161]:
model_lm

Pipeline(steps=[('vectorizer',
                 Pipeline(steps=[('bow',
                                  CountVectorizer(analyzer='char_wb',
                                                  decode_error='ignore',
                                                  strip_accents='unicode'))])),
                ('classifier', ComplementNB())])

In [162]:

# Fit the model
model_lm.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 Pipeline(steps=[('bow',
                                  CountVectorizer(analyzer='char_wb',
                                                  decode_error='ignore',
                                                  strip_accents='unicode'))])),
                ('classifier', ComplementNB())])

In [163]:



# Predict on test data
y_pred = model_lm.predict(X_test)

# Performance of the model
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.842152466367713
